In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, LSTM, Dense
import cv2
import numpy as np
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_styled_landmark(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                             )
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10),thickness=1,circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,44,121),thickness=1,circle_radius=1)
                             )
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76),thickness=1,circle_radius=1),
                             mp_drawing.DrawingSpec(color=(121,44,250),thickness=1,circle_radius=1)
                             )
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66),thickness=1,circle_radius=1),
                             mp_drawing.DrawingSpec(color=(245,66,230),thickness=1,circle_radius=1)
                             )

In [5]:
# frame_count = 0
# fps = 30
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('D:/Research_Paper/Sign_detection/Model/output.mp4', fourcc, fps, (640, 480))

In [6]:
# # Run the loop
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():
#         ret, frame = cap.read()  # Read the frame from the webcam
#         if not ret:
#             break

#         image, results = mediapipe_detection(frame, holistic)  # Detect landmarks

#         draw_styled_landmark(image, results)  # Draw landmarks on the frame

#         out.write(image)  # Write the frame to the video file

#         cv2.imshow('OvenCv Feed', image)  # Display the frame

#         frame_count += 1  # Increment the frame count

#         if frame_count == fps * 5:  # If 5 seconds have passed
#             break

#         if cv2.waitKey(10) & 0xFF == ord('q'):  # If 'q' is pressed
#             break

#     cap.release()  # Release the webcam
#     out.release()  # Release the video writer
#     cv2.destroyAllWindows()  # Close all windows

In [25]:
# import cv2

# def capture_and_save_video(output_file="D:/Research_Paper/Sign_detection/Videos/output.mp4", duration=5):
#     # Open a video capture object (0 indicates the default camera)
#     with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
#         cap = cv2.VideoCapture(0)
#     # Check if the camera opened successfully
#         if not cap.isOpened():
#             print("Error: Could not open camera.")
#             return
        
#         # Get the frames per second (fps) of the camera
#         fps = int(cap.get(cv2.CAP_PROP_FPS))
        
#         # Create a VideoWriter object to save the video
#         fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify codec
#         out = cv2.VideoWriter(output_file, fourcc, fps, (640, 480))  # Adjust resolution as needed
        
#         # Capture video for the specified duration
#         start_time = cv2.getTickCount() / cv2.getTickFrequency()
#         while (cv2.getTickCount() / cv2.getTickFrequency()) - start_time < duration:
#             ret, frame = cap.read()  # Read a frame
#             if not ret:
#                 print("Error: Failed to capture a frame.")
#                 break
#             image, results = mediapipe_detection(frame,holistic)
#             draw_styled_landmark(image,results)
#             # Display the frame
#             cv2.imshow('Video Capture', frame)
        
#             # Write the frame to the output video file
#             out.write(frame)
        
#             # Break the loop if 'q' key is pressed
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break
#         cap.release()
#         out.release()
#         cv2.destroyAllWindows()
# capture_and_save_video()



In [71]:
# def draw_landmark(image, results):
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
#     mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS)
#     mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS)

In [91]:
num_classes = 48

In [92]:
frame_shape=(21,150,150, 3)
model = models.Sequential([

  Conv3D(32, (3,3,3), activation='relu', input_shape=frame_shape),
  MaxPooling3D(pool_size = 2),

  Conv3D(32 , kernel_size = (3,3,3) ,activation = 'relu',),
  MaxPooling3D(pool_size=2),

  Conv3D(64 , kernel_size = (2,2,2) ,activation = 'relu',),
  MaxPooling3D(pool_size=2),
  # Conv3D(64 , kernel_size = (2,2,2) ,activation = 'relu',),
  # MaxPooling3D(pool_size=2),

  #   layers.Conv3D(64 , kernel_size = (3,3,3) ,activation = 'relu',),
  #   layers.MaxPooling3D(pool_size=2),

  #   layers.Conv3D(64 , kernel_size = (3,3,3) ,activation = 'relu',),
  #   layers.MaxPooling3D(pool_size=2),

  Flatten(),
 layers.Reshape((1,18496), input_shape=(18496,)),
 LSTM(500, return_sequences=True),
 LSTM(100, return_sequences= True),
LSTM(50,return_sequences=False),
 Dense(num_classes, activation='softmax')
 ])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 19, 148, 148, 32   2624      
                             )                                   
                                                                 
 max_pooling3d (MaxPooling3  (None, 9, 74, 74, 32)     0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 7, 72, 72, 32)     27680     
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 3, 36, 36, 32)     0         
 g3D)                                                            
                                                                 
 conv3d_2 (Conv3D)           (None, 2, 35, 35, 64)     16448     
                                                        

In [93]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [94]:
model.load_weights('C:/Users/manu/Downloads/model_weights.h5')

In [95]:
# def extract_frames(video_path, num_frames=30, fps=24):
#     cap = cv2.VideoCapture(video_path)
#     if not cap.isOpened():
#         raise ValueError(f"Error opening video: {video_path}")

#     frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     # print("frame_count",frame_count)
#     # print("num_frames",num_frames)
#     if frame_count < 1:
#         raise ValueError("Video has no frames")

#     # Ensure at least one frame is extracted, even for short videos
#     if num_frames < 1:
#         raise ValueError("Number of frames must be at least 1")

#     # print("FPS",int(cap.get(cv2.CAP_PROP_FPS)) )

#     frame_interval = max(1, int(cap.get(cv2.CAP_PROP_FPS)) // num_frames)
#     # print("Frame_interval",frame_interval)
#     frames = []
#     count = 0
#     while cap.isOpened():
#       if count <= 20:
#           # print(count)
#           ret, frame = cap.read()
#           if not ret:
#               #if ret == False
#               # print("Ret value",ret)
#               break
#           if count % frame_interval == 0:
#               frames.append(frame)
#           count += 1
#       elif count > 20:
#           cap.release()
#     if not frames:
#         print("Warning: No frames extracted from video.")
#         return []  # Return an empty list
#     return frames

In [96]:
def preprocess_frames(frames, resize_shape=(150, 150)):

    preprocessed_frames = np.empty((len(frames), *resize_shape, 3))
    for i, frame in enumerate(frames):
        frame = cv2.resize(frame, resize_shape)
        frame = frame / 255.0  # Normalize pixel values to [0, 1]
        preprocessed_frames[i] = frame

    return preprocessed_frames

In [157]:
def capture_video_with_landmarks():
    # Initialize MediaPipe Holistic
    mp_holistic = mp.solutions.holistic
    holistic = mp_holistic.Holistic()

    # Initialize OpenCV VideoCapture
    cap = cv2.VideoCapture(0)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter("D:/Research_Paper/Sign_detection/Videos/output.mp4", fourcc, 20.0, (640, 480))

    # Start time
    start_time = time.time()

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Flip the frame horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe Holistic
        results = holistic.process(rgb_frame)

        # Draw landmarks on the frame
        if results.pose_landmarks:
            draw_styled_landmark(frame, results)

        # Write the frame to the output video
        out.write(frame)

        # Display the frame
        cv2.imshow('frame', frame)

        # Break the loop if the time limit is reached
        if time.time() - start_time > 5:
            break

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoCapture and VideoWriter objects
    cap.release()
    out.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()

    # Close MediaPipe Holistic
    holistic.close()

# Call the function to capture the video with landmarks
capture_video_with_landmarks()


In [152]:
def load_data(data_dir, sequence_length, num_classes):
  X= []
  frames = preprocess_frames(extract_frames("D:/Research_Paper/Sign_detection/Videos/output.mp4"))
  # print("Frames",frames,end = "\n")
            # Create sequence
  for i in range(len(frames) - sequence_length + 1):
      sequence = frames[i:i + sequence_length]
              # print(f"{g}Sequence", sequence,end="\n")
      X.append(sequence)
  print("done")
  # print(X)
  re = np.array(X)
  # print(re)
  print("Shape of X -: ",re.shape)
  return re

In [158]:
data = load_data("D:/Research_Paper/Sign_detection/Videos/",21,48)

done
Shape of X -:  (1, 21, 150, 150, 3)


In [188]:
y = np.array(['you', 'you', 'you', 'you', 'you', 'you', 'your', 'your', 'your', 'why', 'why', 'why', 'why', 'why', 'why', 'why', 'why', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'wrong', 'wrong', 'wrong', 'wrong', 'wrong', 'who', 'who', 'who', 'who', 'who', 'who', 'see you letter', 'see you letter', 'see you letter', 'see you letter', 'see you letter', 'see you letter', 'see you letter', 'see you letter', 'what', 'what', 'what', 'what', 'what', 'what', 'what', 'what', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'want', 'want', 'want', 'want', 'want', 'where', 'where', 'where', 'where', 'where', 'where', 'where', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'thank you', 'thank you', 'thank you', 'thank you', 'thank you', 'thank you', 'thank you', 'thank you', 'when', 'when', 'when', 'when', 'when', 'when', 'when', 'same', 'same', 'same', 'same', 'same', 'same', 'which', 'which', 'which', 'which', 'which', 'which', 'which', 'no', 'no', 'no', 'no', 'no', 'no', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'question', 'question', 'question', 'question', 'question', 'question', 'question', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'more', 'more', 'more', 'more', 'more', 'more', 'more', 'mother', 'mother', 'mother', 'mother', 'mother', 'mother', 'mother', 'mother', 'milk', 'milk', 'milk', 'milk', 'milk', 'milk', 'milk', 'milk', 'nice', 'nice', 'nice', 'nice', 'nice', 'nice', 'nice', 'please', 'please', 'please', 'please', 'please', 'please', 'please', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'go', 'go', 'go', 'go', 'go', 'go', 'go', 'meet', 'meet', 'meet', 'meet', 'meet', 'meet', 'meet', 'meet', 'meet', 'i', 'i', 'i', 'i', 'i', 'i', 'how', 'how', 'how', 'how', 'how', 'how', 'how', 'help', 'help', 'help', 'help', 'help', 'help', 'help', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'like', 'like', 'like', 'like', 'like', 'like', 'like', 'like', 'like', 'like', 'hello', 'hello', 'hello', 'hello', 'hello', 'hello', 'hello', 'learn', 'learn', 'learn', 'learn', 'learn', 'learn', 'learn', 'learn', 'eat', 'eat', 'eat', 'eat', 'eat', 'eat', 'bad', 'bad', 'bad', 'bad', 'bad', 'bad', 'bad', 'busy', 'busy', 'busy', 'busy', 'busy', 'busy', 'finish', 'finish', 'finish', 'finish', 'finish', 'finish', 'finish', 'fine', 'fine', 'fine', 'fine', 'fine', 'fine', 'fine', 'father', 'father', 'father', 'father', 'father', 'father', 'bathroom', 'bathroom', 'bathroom', 'bathroom', 'bathroom', 'bathroom', 'bathroom', 'bathroom', 'bathroom', 'book', 'book', 'book', 'book', 'book', 'book', 'book', 'do not want', 'do not want', 'do not want', 'do not want', 'do not want', 'do not want', 'forget', 'forget', 'forget', 'forget', 'forget', 'forget', 'forget', 'again', 'again', 'again', 'again', 'again', 'again', 'again', 'again'])

# Convert the NumPy array to a list
y_list = y.tolist()
unique_elements = list(set(y))
dict = {element: i + 1 for i, element in enumerate(unique_elements)}

In [191]:
def search_key_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None

In [192]:
search_key_by_value(dict,(np.argmax(model.predict(data))))

1/1 [==============================] - 0s 208ms/step


'fine'